In [16]:
from stage import getStage, fullPrint
import pandas as pd
import datetime
from datetime import date, datetime
import yahoo_fin.stock_info as yf

In [17]:
coef = [0.5590154301570781, 1.0038092967383403, 0.38303606143798996, 1.0362389796691118, 1.0030905657509877, 1.0189883649147726, 0.8666655791915169, 0.9855913108128587, 1.3908949927299494, 0.9804811720459655, -0.19103234802670008, 0.9359023877540643, 0.9908752120306481, 0.808767804685719, 0.9841940289653691]

In [18]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            #print(df.at[date,'MA30Slope'],df.at[date,'RS'])
            #!!!!!!!! or for new params
            #!!!!!!!!
            #!!!!!!!!
            #!!!!!!!!
            if  df.at[date,'MA30Slope'] > coef[9] and df.loc[date,'RS'] > coef[10]:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1
    #print(goodSectorDf.at[date,'Sectors'])
fullPrint(goodSectorDf)

                                                      Sectors
1999-01-08                                                 {}
1999-01-15                                                 {}
1999-01-22                                                 {}
1999-01-29                                                 {}
1999-02-05                                                 {}
1999-02-12                                                 {}
1999-02-19                                                 {}
1999-02-26                                                 {}
1999-03-05                                                 {}
1999-03-12                                                 {}
1999-03-19                                                 {}
1999-03-26                                                 {}
1999-04-09                                                 {}
1999-04-16                                                 {}
1999-04-23                                                 {}
1999-04-

In [4]:
# removeList = []
# nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
# for stock in nasdaqList:
#     try:
#         df = getStage(stock, coef,goodSectorDf)
#         if df.empty:
#             removeList.append(stock)
#     except:
#         removeList.append(stock)

In [4]:
# import pickle
# for removeItem in removeList:
#     nasdaqList.remove(removeItem)
# listOfTickerObject = open('stockData/tickerListpkl', 'wb')
# pickle.dump(nasdaqList, listOfTickerObject)
# listOfTickerObject.close()

In [19]:
# Open a file
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
result = open("Testdata/Realtime/result"+date.today().strftime('%Y-%m-%d')+".txt", "w")
buyList = []
holdList = []
sellList = []
portfolio = []
for symbol in nasdaqList:
    df = getStage(symbol,coef,goodSectorDf)
    shares = 0
    sharesSPY = 0
    if df.empty:
        continue
    if df.iloc[-1].Stage=="Buy":
        print(symbol," buy ",df.iloc[-1].support * coef[7])
        result.write(symbol+" buy " + str(df.iloc[-1].support * coef[7]) + "\n")
        buyList.append(symbol)
        portfolio.append(tuple((symbol, df.iloc[-1].close)))
    elif df.iloc[-1].Stage=="Stage 2":
        print(symbol," hold ",df.iloc[-1].support * coef[7])
        result.write(symbol+" hold " + str(df.iloc[-1].support * coef[7]) + "\n")
        holdList.append(holdList)
        portfolio.append(tuple((symbol, df.iloc[-1].close)))
    elif df.iloc[-1].Stage[0:4]=="Sell":
        print(symbol+" sell")
        result.write(symbol+" sell\n")
        sellList.append(sellList)
result.close()

ACET  buy  16.952171297927666
AGRI sell
ARTW  buy  5.470031962997989
CALM  buy  49.545675645730306
CPLP  hold  14.961276398917793
HBP  hold  9.244846608216589
KROS  buy  50.363714478644084
LAND  buy  35.629127389777835
NFE  buy  37.58059773401939
PNBK  hold  16.163697121357636
SAMG  hold  19.41614957495981
SGML  buy  14.320641482929563
TCDA  buy  8.33810252707411
TH  buy  5.647438229756342
TWNK  buy  21.04237486182778
UG  buy  19.15989560856452


In [13]:
##CURRENT BALANCE
balance = 100
allocation = balance / len(portfolio)
for ticker, close in portfolio:
    print(ticker, allocation / close, close)

ACET 0.369644780861091 20.809999465942383
CALM 0.13104442235653718 58.70000076293945
CPLP 0.41003771846788417 18.760000228881836
HBP 0.7216048595136826 10.65999984741211
KROS 0.12939121267603082 59.45000076293945
LAND 0.20063400265240933 38.34000015258789
NFE 0.17124461277528816 44.91999816894531
SGML 0.495319241762537 15.529999732971191
TCDA 0.8388557937086232 9.170000076293945
TH 1.2863390748783816 5.980000019073486
TWNK 0.33242471457232653 23.139999389648438
UG 0.3378264179913496 22.770000457763672
VTNR 0.7762167316598352 9.90999984741211


In [6]:
for symbol in ["AACG"]:
    df = getStage(symbol,[0.10684256968972883, 1.1421709856192384, 0.27350824408318475, 1.0231080328602544, 1.0760547872425865, 1.2745231615209662, 0.9038554056254413, 1.0764846753295985, 1.2709398376771783, 0.992549883242513, -0.3044455098712273, 0.9467020070162686, 1.0014235121991226, 0.8164237632733558, 0.9867262709280287],goodSectorDf)
    df.to_csv("realtime"+symbol+".csv")

In [7]:
print(buyList)

['AGRI', 'ARTW', 'CPLP', 'HBP', 'PNBK', 'RADA', 'SAMG', 'SND']


In [14]:
getStage("SAMG", coef, goodSectorDf)

,close,Stage,support
2013-06-28,12.000000,,0.000000
2013-07-05,11.590000,volume -0.4776714513556619 1.1421709856192384,0.000000
2013-07-12,12.650000,volume -0.6259541984732824 1.1421709856192384,0.000000
2013-07-19,12.260000,RS -1.1878897336408405 0.27350824408318475,0.000000
2013-07-26,14.060000,volume -0.13807982740021574 1.1421709856192384,0.000000
...,...,...,...
2022-03-04,16.850000,"bad sector XLF{'XLU', 'XLP', 'XLE'}",0.000000
2022-03-11,17.990000,"bad sector XLF{'XLU', 'XLE'}",0.000000
2022-03-18,19.700001,bearish,0.000000
2022-03-25,21.910000,Buy,19.700001
